## LangChain: Models, Prompts and Output Parsers

# Outline¶
Direct API calls to OpenAI
API calls through LangChain:
Prompts
Models
Output parsers

In [ ]:
#!pip install python-dotenv
#!pip install openai


import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key = os.environ['OPENAI_API_KEY']


# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


Chat API : OpenAI
We will do a direct API calls to OpenAI.

In [ ]:


def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [ ]:
get_completion("What is 1+1?")

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)

In [ ]:
response

#until here we are just using regular prompts 

## Chat API : LangChain

We will do the same but using the template capabilities of langchain 

In [ ]:
#!pip install --upgrade langchain

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:

from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)



next shows me that the prompt contains: input variables, output_parser, partial_variables,template formate
and validate_template

In [ ]:
prompt_template.messages[0]

In [ ]:
prompt_template.messages[0].prompt

I can see the prompt now .  To check the input variables that I passed in the template string

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
prompt_template.messages[0].prompt.template


In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""


In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages( style = customer_style, text = customer_email)

In [ ]:
print(type(customer_messages))

In [ ]:
print (type(customer_messages[0]))

In [ ]:
print (customer_messages[0])    

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

Now we can change the customer_style and customer_text and re-use the prompt

In [20]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [21]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)


Translate the text that is delimited by tripple backticks into a style that is 
a polite tone that speaks in English Pirate.
text : ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
 ``` 


In [22]:
service_response = chat(service_messages)
print(service_response.content)

NameError: name 'chat' is not defined

## Output Parsers

Output Parsers.  eX:  I want to receive a json object

In [23]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [24]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
#Here I am telling the model how i want the output to be parsed
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [25]:
# i don't need this because i already did it

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [ ]:
type(response.content)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

This LOOKS LIKE json but it is a string 

In [ ]:
type(response.content)
response.content.get('gift')
# I get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')


## Parse the LLM output string into a Python dictionary

In [28]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [29]:
#This is the schema that I need 
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

#pass the schema into a list
response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [30]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [31]:
format_instructions = output_parser.get_format_instructions()

In [27]:
print(format_instructions)


NameError: name 'format_instructions' is not defined

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')